In [15]:
#primero importamos todos las bibliotecas necesarias
#importamos pytorch
import torch
#importamos el paquete de redes neuronales
import torch.nn as nn
import torch.nn.functional as F
#importamos el paquete que permite cargar los datos
from torch.utils.data import Dataset, Subset
import torchvision
from torchvision import datasets
from torchvision.datasets import ImageFolder
#importamos el paquete que permite el autoentrenamiento del modelo
from torchvision.transforms import ToTensor
#importamos los dataloaders
from torch.utils.data import DataLoader
#importamos la función de optimización
from torch import optim
#importamos el paquete para el entrenamiento de la red
from torch.autograd import Variable

In [5]:
#establecemos el dispositivo a usar (cpu o gpu)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [10]:
#cargamos las imágenes del directorio que emplearemos para el entrenamiento
OCT = ImageFolder(root='Datos/Processed Data/OCT')

In [17]:
#realizamos la división de las imágenes de entrenamiento en un 80% para train y un 20% para validation
n = len(OCT)
n_val = int(0.2*n)
OCT_train = Subset(OCT, range(n_val,n))
OCT_val = Subset(OCT, range(n_val))

In [19]:
#comprobamos que las dimensiones del conjunto de train y validation son las adecuadas
print(f'Longitud del conjunto de train: {len(OCT_train)}')
print(f'Longitud del conjunto de validation: {len(OCT_val)}')

Longitud del conjunto de train: 91
Longitud del conjunto de validation: 22


In [20]:
#definimos los dos conjuntos de test
Samsung_test = ImageFolder(root='Datos/Processed Data/Samsung')
iPhone_test = ImageFolder(root='Datos/Processed Data/iPhone')

In [21]:
#y observamos sus dimensiones
print(f'Longitud del conjunto de test de Samsung: {len(Samsung_test)}')
print(f'Longitud del conjunto de test de iPhone: {len(iPhone_test)}')

Longitud del conjunto de test de Samsung: 93
Longitud del conjunto de test de iPhone: 99
